# Getting Started with TensorLogic

TensorLogic is a neural-symbolic AI framework that unifies logical reasoning and tensor computation. Based on Pedro Domingos' Tensor Logic paper (arXiv:2510.12269), it implements the mathematical equivalence between logical rules and Einstein summation.

## What You'll Learn

1. Installation and setup
2. Basic logical operations (AND, OR, NOT, IMPLIES)
3. Creating your first knowledge graph
4. Using the backend abstraction

## Installation

```bash
# Clone the repository
git clone https://github.com/your-username/TensorLogic.git
cd TensorLogic

# Install with uv (recommended)
uv sync

# Or with pip
pip install -e .
```

## Imports

TensorLogic provides clean top-level exports for all core functionality.

In [ ]:
# Core imports from TensorLogic
from tensorlogic import (
    # High-level API
    quantify,
    reason,
    # Backend
    create_backend,
    TensorBackend,
    # Logical operations
    logical_and,
    logical_or,
    logical_not,
    logical_implies,
    # Quantifiers
    exists,
    forall,
)

import numpy as np

print("TensorLogic loaded successfully!")

## Backend Abstraction

TensorLogic uses a backend abstraction that supports MLX (Apple Silicon) and NumPy. The `create_backend()` function automatically selects the best available backend.

In [ ]:
# Create a backend (MLX if available, otherwise NumPy)
backend = create_backend()
print(f"Backend: {type(backend).__name__}")

# You can also explicitly select a backend
numpy_backend = create_backend("numpy")
print(f"NumPy Backend: {type(numpy_backend).__name__}")

## The Tensor-Logic Mapping

The core insight of TensorLogic is that logical operations map directly to tensor operations:

| Logical Operation | Tensor Implementation | Interpretation |
|-------------------|----------------------|----------------|
| AND | Hadamard product (element-wise multiply) | Both conditions must hold |
| OR | Element-wise maximum | At least one condition holds |
| NOT | Complement (1 - x) | Negation |
| Implication (P → Q) | max(1 - P, Q) | Classical material conditional |
| Existential quantifier | Summation over axis + step | "There exists" |
| Universal quantifier | Product over axis + threshold | "For all" |

## Basic Logical Operations

Let's explore each operation with simple examples.

In [ ]:
# Create boolean tensors (values in {0, 1})
a = np.array([1.0, 1.0, 0.0, 0.0])  # True, True, False, False
b = np.array([1.0, 0.0, 1.0, 0.0])  # True, False, True, False

print("Input:")
print(f"  a = {a}")
print(f"  b = {b}")

In [ ]:
# Logical AND: a * b (Hadamard product)
result_and = logical_and(a, b, backend=backend)
print(f"a AND b = {result_and}")
print("  (Both must be True)")

In [ ]:
# Logical OR: max(a, b)
result_or = logical_or(a, b, backend=backend)
print(f"a OR b = {result_or}")
print("  (At least one must be True)")

In [ ]:
# Logical NOT: 1 - a
result_not = logical_not(a, backend=backend)
print(f"NOT a = {result_not}")
print("  (Complement)")

In [ ]:
# Logical IMPLIES: max(1-a, b)
# a -> b is equivalent to "if a then b"
result_implies = logical_implies(a, b, backend=backend)
print(f"a -> b = {result_implies}")
print("  (Implication: max(1-a, b))")
print("  Note: Implication is True when a is False OR b is True")

## Creating Your First Knowledge Graph

A knowledge graph represents entities and their relationships as tensors. Let's create a simple family tree.

In [ ]:
# Define entities
entities = ["Alice", "Bob", "Carol", "David"]
n = len(entities)
entity_to_idx = {name: i for i, name in enumerate(entities)}

print("Entities:")
for name, idx in entity_to_idx.items():
    print(f"  [{idx}] {name}")

In [ ]:
# Create Parent relation tensor (n x n matrix)
# Parent[i, j] = 1.0 means entity i is parent of entity j
parent = np.zeros((n, n), dtype=np.float32)

# Add facts: Alice and Bob are parents of Carol and David
parent_facts = [
    ("Alice", "Carol"),
    ("Alice", "David"),
    ("Bob", "Carol"),
    ("Bob", "David"),
]

for p, c in parent_facts:
    parent[entity_to_idx[p], entity_to_idx[c]] = 1.0

print("Parent Relation Matrix:")
print(f"{'':>8}", end="")
for name in entities:
    print(f"{name:>7}", end="")
print()

for i, row_name in enumerate(entities):
    print(f"{row_name:>8}", end="")
    for j in range(n):
        print(f"{parent[i, j]:>7.0f}", end="")
    print()

## Querying the Knowledge Graph

Now let's query our knowledge graph using logical operations.

In [ ]:
# Query: Is Alice a parent of Carol?
alice_idx = entity_to_idx["Alice"]
carol_idx = entity_to_idx["Carol"]

is_parent = parent[alice_idx, carol_idx]
print(f"Is Alice parent of Carol? {bool(is_parent)}")

In [ ]:
# Query: Does Alice have any children? (EXISTS)
# exists y: Parent(Alice, y)
alice_row = parent[alice_idx, :]  # Get Alice's row

has_children = exists(alice_row.reshape(1, -1), axis=1, backend=backend)
print(f"Does Alice have children? {bool(has_children[0])}")

In [ ]:
# Query: Who has children?
# For each person x, check: exists y: Parent(x, y)
has_children_all = exists(parent, axis=1, backend=backend)

print("Who has children?")
for i, name in enumerate(entities):
    result = "Yes" if has_children_all[i] > 0 else "No"
    print(f"  {name}: {result}")

## Using the High-Level API

The `quantify()` and `reason()` functions provide an einops-style pattern language for more readable queries.

In [ ]:
# Using quantify() for pattern-based queries
result = quantify(
    "exists x: P(x)",
    predicates={"P": has_children_all},
    backend=backend,
)

print(f"Does anyone have children? {bool(result)}")

In [ ]:
# Using reason() with temperature control
result = reason(
    "exists y: P(y)",
    predicates={"P": parent[alice_idx, :]},
    bindings={"y": np.arange(n)},
    temperature=0.0,  # Deductive mode (exact logic)
    backend=backend,
)

result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
print(f"Does Alice have children (deductive)? {bool(result_val)}")

## Summary

In this notebook, you learned:

1. **Backend Abstraction**: TensorLogic uses a protocol-based backend that supports MLX and NumPy
2. **Logical Operations**: AND, OR, NOT, IMPLIES map to tensor operations
3. **Knowledge Graphs**: Relations are represented as tensors (matrices)
4. **Quantifiers**: EXISTS and FORALL aggregate over tensor dimensions
5. **High-Level API**: `quantify()` and `reason()` provide readable pattern syntax

## Next Steps

- **02_knowledge_graphs.ipynb**: Multi-hop reasoning (grandparent, aunt/uncle inference)
- **03_compilation_strategies.ipynb**: Different semantic interpretations
- **04_temperature_control.ipynb**: Deductive vs analogical reasoning